In [6]:
#import necessary libraries
import numpy as np
import pandas as pd
from pandas import read_csv
from glob import glob
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
#INPUT PARAMETERS
dischargePowerCap = float(100) #max discharge power capacity in kW
chargePowerCap = float(100) #max charge power capacity in kW
dischargeEnergyCap = float(200) #max discharge energy capacity in kWh
RTefficiency = 0.85 #AC-AC roundtrip efficiency
maxDailyThroughput = float(200) #max daily discharge throughput in kWh
chargeTime = int(dischargeEnergyCap/chargePowerCap) #hour
dischargeTime = int(dischargeEnergyCap/dischargePowerCap) #hr

In [9]:
#read in all csv files at once from directory
#feed in your filenames here for HOURLY LBMP
path = Path.cwd() / "2017_NYISO_LBMPs"
filenames = path.glob('**/*.csv')
#get list of dataframes for each day
dataframes= [pd.read_csv(f, header= 0, index_col=0) for f in filenames]
#input ZONE name here
zone = 'N.Y.C.'

In [10]:
#initializing variables:
storedEnergy = 0
dischargedEnergy = 0
dfName_list = []
b = 0 # starting day 1
q = 1
for x in range(1,len(dataframes)+1):
     # specify the desired zone
    df1 = dataframes[b]
    df1 = df1[df1.Name == zone]
    table = df1.sort_values(by='LBMP ($/MWHr)', ascending=True) # sort hourly LBMP with single day
    times = table.index.values
    # add columns to dataframe containing desired output data
    df1 = df1.assign(Battery_Power_Output=np.nan, SOE=np.nan, Hourly_Revenue_Generation=np.nan,
                     Hourly_Charging_Cost=np.nan, Throughput=np.nan, IntIndex=np.nan)

    # initialize arrays of data that will be inserted into our dataframe
    SOE_array = np.zeros(len(table))
    Battery_Output_array = np.zeros(len(table))
    Hourly_Revenue_array = np.zeros(len(table))
    Hourly_Charging_Cost = np.zeros(len(table))
    Throughput_array = np.zeros(len(table))
    # scale our LBMP values to kW:
    LBMPvals = -df1['Marginal Cost Congestion ($/MWHr)'].values
    # put in a numerical index
    l = [i for i in range(24)]
    df1['IntIndex'] = l

    # sort our given data by price (from lowest to highest price)
    table = df1.sort_values(by='LBMP ($/MWHr)', ascending=True)

    # get our sorted time values into an array
    times = table.index.values

    # get the integer indexes for highest and lowest prices
    chargeIndex = table.iloc[0:2]
    chargeIndex = chargeIndex['IntIndex'].sort_values(ascending=True)
    dischargeIndex = table.iloc[-2:]
    dischargeRef = table.iloc[-1]
    dischargeRef = dischargeRef['IntIndex']#create a reference list ordered by price value to keep track of order by price
    dischargeIndex = dischargeIndex['IntIndex'].sort_values(ascending=True)

    num = len(table)
    # create a loop to go through each hour of the day

    w = 0
    y = 1

    # create an iterator for charge index
    r = 0
    # create an iterator of discharge index
    k = 0
    for z in range(1, len(LBMPvals)):
        print('Z:', z)

        # create an integer index for each day we iterate through
        intIndex = y
        print('IntIndex: ', intIndex)
        print('ChargeIndex: ', chargeIndex[r])
        print('dischargeIndex: ', dischargeIndex[k])

        # conditions where we can charge the battery
        #charge the battery if it is one of the cheapest times and the battery is not already charged:
        if ((intIndex == chargeIndex[r]) & (SOE_array[y-1]<dischargeEnergyCap)):
            # if SOE_array[y-1]<dischargeEnergyCap:
            SOE_array[y] = SOE_array[y - 1] + (chargeTime * chargePowerCap)
            Battery_Output_array[y] = 0  # we do not discharge while charging
            Hourly_Revenue_array[y] = 0  # no revenue is generated while charging
            chargeEnergy = chargePowerCap
            Hourly_Charging_Cost[y] = LBMPvals[y] * chargeTime * 0.001 *chargeEnergy
            Throughput_array[y] = 0  # no discharge throughput while charging
            if r == 0:
                r += 1
            print('CHARGE')
            print(SOE_array[y], Battery_Output_array[y], Hourly_Revenue_array[y], Hourly_Charging_Cost[y],
                  Throughput_array[y])
        # conditions where we discharge the battery
        #charge the battery if the time if LBMP high, the battery has charge, and we have not yet reached our max daily discharging throughput
        elif ((intIndex == dischargeIndex[k]) & (SOE_array[y-1]>0) & (Throughput_array[y]<maxDailyThroughput)):

            if (intIndex == dischargeRef):
                dischargeEnergy = dischargePowerCap
                loss = 0
            else:
                val = dischargePowerCap * RTefficiency #take into account roundtrip efficiency (get less output power than input)
                dischargeEnergy = val
                loss = dischargePowerCap * (1-RTefficiency) #losses that we experience when discharging battery

            SOE_array[y] = SOE_array[y - 1] - dischargeEnergy - loss
            Battery_Output_array[y] = dischargeEnergy
            Hourly_Revenue_array[y] = LBMPvals[y] * dischargeTime *0.001 *dischargeEnergy
            Hourly_Charging_Cost[y] = 0  # no cost for charging while batterry is outputting
            Throughput_array[y] = dischargeEnergy

            if k == 0:
                k += 1
            print('DISCHARGE')
            print(SOE_array[y], Battery_Output_array[y], Hourly_Revenue_array[y], Hourly_Charging_Cost[y],
                  Throughput_array[y])
        # conditions when we are not charging or discharging (based on LBMP price)
        #so the conditions remain the same as the previous value in the array
        else:
            prevVal = SOE_array[y - 1]
            SOE_array[y] = prevVal
            Battery_Output_array[y] = 0
            Hourly_Revenue_array[y] = 0
            Hourly_Charging_Cost[y] = 0
            Throughput_array[y] = 0  # no discharge throughput while charging
            print('ELSE')
            print(SOE_array[y], Battery_Output_array[y], Hourly_Revenue_array[y], Hourly_Charging_Cost[y],
                  Throughput_array[y])

        print('#####################################################')

        w += 1
        y += 1

    df1['Battery_Power_Output'] = Battery_Output_array
    df1['SOE'] = SOE_array
    df1['Hourly_Revenue_Generation'] = Hourly_Revenue_array
    df1['Hourly_Charging_Cost'] = Hourly_Charging_Cost
    df1['Throughput'] = Throughput_array
    dfName = 'new' + filenames[b]
    dfName_list.append(dfName)
    df1.to_csv(dfName, sep= ',')
    b += 1
    q += 1


Z: 1
IntIndex:  1
ChargeIndex:  5
dischargeIndex:  16
ELSE
0.0 0.0 0.0 0.0 0.0
#####################################################
Z: 2
IntIndex:  2
ChargeIndex:  5
dischargeIndex:  16
ELSE
0.0 0.0 0.0 0.0 0.0
#####################################################
Z: 3
IntIndex:  3
ChargeIndex:  5
dischargeIndex:  16
ELSE
0.0 0.0 0.0 0.0 0.0
#####################################################
Z: 4
IntIndex:  4
ChargeIndex:  5
dischargeIndex:  16
ELSE
0.0 0.0 0.0 0.0 0.0
#####################################################
Z: 5
IntIndex:  5
ChargeIndex:  5
dischargeIndex:  16
CHARGE
200.0 0.0 0.0 0.20600000000000002 0.0
#####################################################
Z: 6
IntIndex:  6
ChargeIndex:  6
dischargeIndex:  16
ELSE
200.0 0.0 0.0 0.0 0.0
#####################################################
Z: 7
IntIndex:  7
ChargeIndex:  6
dischargeIndex:  16
ELSE
200.0 0.0 0.0 0.0 0.0
#####################################################
Z: 8
IntIndex:  8
ChargeIndex:  6
dischargeIn

TypeError: 'generator' object is not subscriptable